# 🤖 Building Autonomous Agents: Adding Tools

Welcome to the next step in building autonomous agents! In this notebook, we'll focus on expanding an LLM's capabilities by giving it access to tools.

Autonomous agents need to interact with their environment, and tools are the foundation that allows LLMs to take actions beyond just generating text. By providing tools to a language model, we enable it to perform specific tasks like searching for information, making calculations, or accessing external systems.

## Objectives:
- Understand the role of tools in autonomous agents
- Create and register tools for LLM use
- Implement a basic tool-using agentic system with LangGraph
- Observe how tools transform a passive LLM into an active agent

Throughout this notebook, we'll build a simple agent that can use tools to answer questions to extend the models capabilities. This forms the foundation for the more advanced agent architectures we'll explore in later notebooks.

Let's begin by setting up our environment and defining our first set of tools! 🚀

In [ ]:
# Import dependencies. 

import boto3
import json

# Initialize the Bedrock client
session = boto3.Session()
bedrock = session.client(service_name='bedrock-runtime')

print("✅ Setup complete!")

In the previous lab, we added memory to that chat bot. Let's extend this chat bot to use tools. In a similar fashion, we'll implement this with vanilla Python and then show you how it would look in LangGraph.

We also made a case for owning our own types as abstraction layers for all things that touch LLMs. Creating abstractions is the best way to make 2-way door decisions if you decide you, for example want to use crew.ai or pydantic ai instead of LangGraph. The less you abstract away, the harder the refactor will be if you change your mind.

## Reuse what we wrote in the previous lab. 
We've pushed the types and converters from the previous lab into a common folder (similar to what exists in the agent platform). We'll just import them for reuse here.

In [ ]:
from agentic_platform.core.models.prompt_models import BasePrompt
from agentic_platform.core.models.memory_models import Message, SessionContext, ToolResult, ToolCall
from agentic_platform.core.models.llm_models import LLMResponse, LLMResponse

from typing import Dict, Any, Optional, List, Type, Callable

# Clients.
class MemoryClient:
    """Manages conversations"""
    def __init__(self):
        self.conversations: Dict[str, SessionContext] = {}

    def upsert_conversation(self, conversation: SessionContext) -> bool:
        self.conversations[conversation.session_id] = conversation

    def get_or_create_conversation(self, conversation_id: str=None) -> SessionContext:
        return self.conversations.get(conversation_id, SessionContext()) if conversation_id else SessionContext()
    
memory_client: MemoryClient = MemoryClient()

# Create Tool Specification
For our tool calls we need a tool specification. Each API provider has a slightly different tool spec making the need for abstraction necessary. We'll define our own below. 

**Note**: With model context protocol or MCP (discussed in module 4), there's an initiative to standardize these specifications moving forward which would be a huge leap for the community. We'll base our tool spec off MCPs, but implement it in a way that pydantic models can be used to generate the tool specs. 

In [ ]:
from agentic_platform.core.models.tool_models import ToolSpec

ToolSpec??

# Reuse Bedrock Call
The call_bedrock function from the previous lab was almost complete. We need to modify the LLMRequest object to accept our new ToolSpecs. Lets do that now

In [ ]:

# Import Converse API converter to convert the raw JSON into our own types.
from agentic_platform.core.converter.llm_request_converters import ConverseRequestConverter
from agentic_platform.core.converter.llm_response_converters import ConverseResponseConverter
from agentic_platform.core.models.llm_models import LLMRequest, LLMResponse

# Helper function to call Bedrock. Passing around JSON is messy and error prone.
def call_bedrock(request: LLMRequest) -> LLMResponse:
    kwargs: Dict[str, Any] = ConverseRequestConverter.convert_llm_request(request)
    # Call Bedrock
    converse_response: Dict[str, Any] = bedrock.converse(**kwargs)
    # Get the model's text response
    return ConverseResponseConverter.to_llm_response(converse_response)

Now let's test out our new types! We'll do two tests.
1. We'll generate structured output (as a pydantic model) from our tool calling. 
2. We'll use the model to pick which tool to use

To get structured output, we can use force_tool from the converse API to force the tool use. This will give us our structured output

In [ ]:
from pydantic import BaseModel
# Create a pydantic model we want to use for structured output.
class WeatherReport(BaseModel):
    temperature: float
    condition: str

# Then create a ToolSchema instance (not a class inheriting from it)
weather_tool = ToolSpec(
    model=WeatherReport,
    name="weather_report",
    description="Useful for getting the weather report for a location."
)

# Create a list of tools we want to use. for this example we only have one.
tools: List[ToolSpec] = [weather_tool]

# Create a prompt with our system and user messages.
prompt: BasePrompt = BasePrompt(
    system_prompt="You are a weather reporter. You are given a location and you need to report the weather.",
    user_prompt="What is the weather in San Francisco?",
)

request: LLMRequest = LLMRequest(
    system_prompt = prompt.system_prompt,
    messages = [ Message.from_text(role="user", text=prompt.user_prompt) ],
    model_id = prompt.model_id,
    hyperparams = prompt.hyperparams,
    tools = tools,
    force_tool = "weather_report"
)

response: LLMResponse = call_bedrock(request)

# We know that there's only one tool invocation.
tool_invocation: ToolCall = response.tool_calls[0]

# Using Pydantics, model_validate(), we can validate the tool invocation is in the correct format.
my_weather_report: WeatherReport = WeatherReport.model_validate(tool_invocation.arguments)

print(my_weather_report)


Great! Using our types, we can easily pass in single tools, force the tool use and get back structured output as pydantic models! Passing around JSON is messy. This gives us a very clean way to work with LLMs without passing around raw text or arbitrary json blobs.

Next, let's try a more complicated tool us pattern. Let's create two python functions we can wrap. 

As a first step let's create another tool and add it to our tool list. 

In [ ]:
from enum import Enum
from pydantic import BaseModel

class Operation(str, Enum):
    add = "add"
    subtract = "subtract"
    multiply = "multiply"
    divide = "divide"

class Calculator(BaseModel):
    operation: Operation
    x: float
    y: float

# Create a tool spec for our calculator.
calculator_tool: ToolSpec = ToolSpec(
    model=Calculator,
    name="calculate",
    description="Perform a mathematical calculation."
)

# Now we have two tools to choose from!
tools: List[ToolSpec] = [weather_tool, calculator_tool]

# Create a prompt with our system and user messages.
class AgentPrompt(BasePrompt):
    system_prompt: str = "You are a helpful assistant. You are given tools to help you accomplish your task. You can choose to use them or not."
    user_prompt: str = "{user_message}"

In [ ]:
# Pass in our user message to the Base Prompt.
inputs: Dict[str, Any] = {"user_message": "What is 2 + 2?"}
prompt: AgentPrompt = AgentPrompt(inputs=inputs)

request: LLMRequest = LLMRequest(
    system_prompt = prompt.system_prompt,
    messages = [ Message.from_text(role="user", text=prompt.user_prompt) ],
    model_id = prompt.model_id,
    hyperparams = prompt.hyperparams,
    tools = tools
)

# call bedrock
response: LLMResponse = call_bedrock(request)

# Get the tool incocation
tool_invocations: List[ToolCall] = response.tool_calls

print(tool_invocations)

Nice! The model is correctly picking the right tool and filling in the tool spec definition. 

### Take a Pause
In the code above, we created types for almost everything that touches Bedrock from our LLMResponse to our tool specifications or ToolSpecs. This is great progress! 

### What's next? 
* Update our Message object to handle tool calls
* Tie the actual tool implementations to the ToolSpec objects and put the tools in our agent.

First lets start by creating our tool implementations and adding them to the ToolSpecs we already created

In [ ]:
from typing import Dict, Any

# This function takes in the arguments defined as a pydantic model.
def weather_report(weather_input: WeatherReport) -> str:
    """Weather report tool"""    
    # NOTE: In a real implementation, we'd call an external API here.
    return "The weather is sunny and 70 degrees."

def handle_calculation(args: Calculator) -> float:
    """Process the calculation request and return a result"""
    x = args.x
    y = args.y

    operator: Operation = args.operation
    
    if operator == 'add':
        result = x + y
    elif operator == 'subtract':
        result = x - y
    elif operator == 'multiply':
        result = x * y
    elif operator == 'divide':
        result = x / y if y != 0 else 'Error: Division by zero'

    return result

In [ ]:
# Add the functions to the tool schemas. In practice you'd want to do this on the tool spec creation instead 
# of mutating the tool spec after creation. In subsequent labs, we'll learn how to use decorators to do this.
weather_tool.function = weather_report
calculator_tool.function = handle_calculation

# Add Tool Use to our Agent
Let's take our chat bot implementation that has memory and extend it to do tool use.

We'll use the tool definition from the first notebook for performing math or making up weather reports.

In [ ]:
from pydantic import BaseModel

# Import our agent request and response types.
from agentic_platform.core.models.api_models import AgenticRequest, AgenticRequest, AgenticResponse
from agentic_platform.core.models.memory_models import TextContent

# Lets reuse this from the previous lab.
memory_client: MemoryClient = MemoryClient()

# Create a prompt with our system and user messages.
class AgentPrompt(BasePrompt):
    system_prompt: str = "You are a helpful assistant. You are given tools to help you accomplish your task. You can choose to use them or not."
    user_prompt: str = "{user_message}"


class ToolCallingAgent:
    # This is new, we're adding tools in the constructor to bind them to the agent.
    # Don't get too attached to this idea, it'll change as we get into MCP.
    def __init__(self, tools: List[ToolSpec], prompt: BasePrompt):
        self.tools: List[ToolSpec] = tools
        self.conversation: SessionContext = SessionContext()
        self.prompt: BasePrompt = prompt

    def call_llm(self) -> LLMResponse:
        # Create LLM request
        request: LLMRequest = LLMRequest(
            system_prompt=self.prompt.system_prompt,
            messages=self.conversation.get_messages(),
            model_id=self.prompt.model_id,
            hyperparams=self.prompt.hyperparams,
            tools=self.tools
        )

        # Call the LLM.
        response: LLMResponse = call_bedrock(request)
        # Append the llms response to the conversation.
        self.conversation.add_message(Message(
            role="assistant",
            text=response.text,
            tool_calls=response.tool_calls
        ))
        # Return the response.
        return response
    
    def execute_tools(self, llm_response: LLMResponse) -> List[ToolResult]:
        """Call tools and return the results."""
        # It's possible that the model will call multiple tools.
        tool_results: List[ToolResult] = []
        # Iterate over the tool calls and call the tool.
        for tool_invocation in llm_response.tool_calls:
            # Get the tool spec for the tool call.
            tool: ToolSpec = next((t for t in self.tools if t.name == tool_invocation.name), None)
            # Call the tool.
            input_data: BaseModel = tool.model.model_validate(tool_invocation.arguments)
            function_result: str = str(tool.function(input_data))
            tool_response: ToolResult = ToolResult(
                id=tool_invocation.id,
                content=[TextContent(text=function_result)],
                isError=False
            )

            print(f"Tool response: {tool_response}")

            # Add the tool result to the list.
            tool_results.append(tool_response)

        # Add the tool results to the conversation
        message: Message = Message(role="user", tool_results=tool_results)
        self.conversation.add_message(message)
        
        # Return the tool results even though we don't use it.
        return tool_results
    
    def invoke(self, request: AgenticRequest) -> AgenticResponse:
        # Get or create conversation
        self.conversation = memory_client.get_or_create_conversation(request.session_id)
        # Add user message to conversation
        self.conversation.add_message(request.message)

        # Keep calling LLM until we get a final response
        while True:
            # Call the LLM
            response: LLMResponse = self.call_llm()
            
            # If the model wants to use tools
            if response.stop_reason == "tool_use":
                # Execute the tools
                self.execute_tools(response)
                # Continue the loop to get final response
                continue
            
            # If we get here, it's a final response 
            break

        # Save updated conversation
        memory_client.upsert_conversation(self.conversation)

        # Return our own type.
        return AgenticResponse(
            message=self.conversation.messages[-1], # Just return the last message
            session_id=self.conversation.session_id
        )

# Call Agent
Now that we have a tool calling agent, lets invoke it.

In [ ]:
# Helper to construct request
def construct_request(user_message: str, conversation_id: str=None) -> AgenticRequest:
    return AgenticRequest.from_text(
        text=user_message, 
        **{'session_id': conversation_id}
    )

agent: ToolCallingAgent = ToolCallingAgent(
    tools=[weather_tool, calculator_tool],
    prompt=AgentPrompt()
)

# Invoke the agent
user_message: str = "What is the weather in SF?"
request: AgenticRequest = construct_request(user_message)
response: AgenticResponse = agent.invoke(request)

# Print the response
print(response.message.model_dump_json(indent=2))

Nice! Because we're updating the conversation and outputting it to our memory client, we can use the conversation id to get the conversation. Conveniently, the conversation object kind of acts as a trace so we can view it to see what our agent did

In [ ]:
# Check out the conversation in the memory client.
convo: SessionContext = memory_client.get_or_create_conversation(response.session_id)

for message in convo.messages:
    print(message)

# Success!
We were able to build an agent that does tool calling in ~100 lines of code using the abstraction layers we put in place.

# What did we just do?
You just successfully built your first autonomous agent! In a modern way (as of 2025), you just built what is commonly referred as a ReACT agent. 

ReACT was the first major orchestration technique that gained traction. With ReACT an LLM is given a prompt describing tools it has access to and a scratch pad for dumping intermediate step results. ReACT is inspired by human abilities to “reason” and “act” to complete tasks. 

When ReACT first came out, function calling didn't exist yet. So the original prompt that looked like this:
```
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: {input}
Thought:{agent_scratchpad}
```

This prompt was designed to make it easier to regex out API calls to do multi-turn conversations and actions. You can now pretty much do that out of the box with the converse API and a while loop (which is what we did). The thoughts and observation chain is now handled by the messages passed into the converse API and you really only need to make small tweaks do ReACT with the converse API directly. 

tl;dr: You pretty much already implemented ReACt in the previous workshop and built your first full fledged agent!



# Next Steps
With this approach, we can add more and more tools to our agent. For example, if you want the agent to do a search online, we could just define a new ToolSpec, add it to our agent and it would work as is!

Next, we'll explore how to add retrieval to our agent 🚀